In [74]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import folium
from folium.plugins import TimestampedGeoJson, HeatMapWithTime

pd.options.display.max_colwidth = 200

In [75]:
file_name = 'results/greedy_50_vehicles_100_bookings_linear.csv.gz'

data = pd.read_csv(file_name, sep=';',
                   # convert column to dict
                   converters={'details': lambda v: eval(v)})

print(f'Data shape {data.shape}')

# Use only 10% of the whole dataset
data = data[:int(data.shape[0] * 0.1)]


data.head()

Data shape (195663, 9)


,clock_time,object_type,uuid,itinerary_id,from_state,to_state,lon,lat,details
0,0,vehicle,0,NaN,offline,idling,-73.99715,40.71196,{}
1,0,vehicle,1,NaN,offline,idling,-73.98232,40.76953,{}
2,0,vehicle,2,NaN,offline,idling,-73.98243,40.78500,{}
3,0,vehicle,3,NaN,offline,idling,-74.00742,40.72601,{}
4,0,vehicle,4,NaN,offline,idling,-73.97801,40.75811,{}


In [46]:
details = data.details.apply(pd.Series)

details[~data.itinerary_id.isna()].head()

,dropoff,eta,pickup,dst,route_duration,route_distance,trip_duration,trip_distance,vid,stop
51,NaN,14.0,0.0,"{'lat': 40.76657, 'lon': -73.9569}",14.0,1.301,0.0,0.000,NaN,NaN
52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN
53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN
54,NaN,14.0,0.0,"{'lat': 40.76657, 'lon': -73.9569}",14.0,1.301,1.0,0.093,NaN,change_dest
55,NaN,14.0,0.0,"{'lat': 40.76657, 'lon': -73.9569}",14.0,1.219,0.0,0.000,NaN,NaN


In [77]:
# history of one vehicle

vehicle_uuid = data[data.object_type == 'vehicle'].uuid.values[0]

vehicle_changes = data[(data.uuid == vehicle_uuid) & (data.object_type == 'vehicle')]

vehicle_changes

,clock_time,object_type,uuid,itinerary_id,from_state,to_state,lon,lat,details
0,0,vehicle,0,NaN,offline,idling,-73.99715,40.71196,{}
371,27,vehicle,0,e154ea24f8e2439d844989104fcaa870,idling,moving_to,-73.99715,40.71196,"{'eta': 11, 'pickup': 11, 'dst': {'lat': 40.71689, 'lon': -73.99567}, 'route_duration': 10, 'route_distance': 0.695, 'trip_duration': 0, 'trip_distance': 0.0}"
394,28,vehicle,0,e154ea24f8e2439d844989104fcaa870,moving_to,idling,-73.99780,40.71205,"{'stop': 'change_dest', 'eta': 11, 'pickup': 11, 'dst': {'lat': 40.71689, 'lon': -73.99567}, 'route_duration': 10, 'route_distance': 0.695, 'trip_duration': 1, 'trip_distance': 0.069}"
395,28,vehicle,0,e154ea24f8e2439d844989104fcaa870,idling,moving_to,-73.99780,40.71205,"{'eta': 11, 'pickup': 11, 'dst': {'lat': 40.71689, 'lon': -73.99567}, 'route_duration': 9, 'route_distance': 0.634, 'trip_duration': 0, 'trip_distance': 0.0}"
416,29,vehicle,0,e154ea24f8e2439d844989104fcaa870,moving_to,idling,-73.99797,40.71266,"{'stop': 'change_dest', 'eta': 11, 'pickup': 11, 'dst': {'lat': 40.71689, 'lon': -73.99567}, 'route_duration': 9, 'route_distance': 0.634, 'trip_duration': 1, 'trip_distance': 0.07}"
...,...,...,...,...,...,...,...,...,...
19386,339,vehicle,0,e154ea24f8e2439d844989104fcaa870,moving_to,idling,-73.99567,40.71689,"{'stop': 'arrived', 'eta': 11, 'pickup': 11, 'dst': {'lat': 40.71689, 'lon': -73.99567}, 'route_duration': 1, 'route_distance': 0.0, 'trip_duration': 1, 'trip_distance': 0.0}"
19419,339,vehicle,0,e154ea24f8e2439d844989104fcaa870,idling,moving_to,-73.99567,40.71689,"{'eta': 11, 'pickup': 11, 'dst': {'lat': 40.71689, 'lon': -73.99567}, 'route_duration': 1, 'route_distance': 0.0, 'trip_duration': 0, 'trip_distance': 0.0}"
19471,340,vehicle,0,e154ea24f8e2439d844989104fcaa870,moving_to,idling,-73.99567,40.71689,"{'stop': 'arrived', 'eta': 11, 'pickup': 11, 'dst': {'lat': 40.71689, 'lon': -73.99567}, 'route_duration': 1, 'route_distance': 0.0, 'trip_duration': 1, 'trip_distance': 0.0}"
19503,340,vehicle,0,e154ea24f8e2439d844989104fcaa870,idling,moving_to,-73.99567,40.71689,"{'eta': 11, 'pickup': 11, 'dst': {'lat': 40.71689, 'lon': -73.99567}, 'route_duration': 1, 'route_distance': 0.0, 'trip_duration': 0, 'trip_distance': 0.0}"


In [73]:
m = folium.Map(location=[40.76953, -73.98232], zoom_start=12)

now = datetime.now()
features = []

prev_pos = None
for item in vehicle_changes.itertuples():
    p1 = item.lon, item.lat
    
    folium.Circle(
        radius=10,
        location=[p1[1], p1[0]],
        color='green',
        fill=True,
    ).add_to(m)

#     if prev_pos:
#         p1 = prev_pos
        
#     p2 = item.lon, item.lat

#     tm = str(now + timedelta(minutes=(item.clock_time)))

#     color = 'brown'
# #     if s['details'].get('pickup'):
#     if item.details.get('pickup') is not None:
#         color = 'green'
        
        
#         folium.Circle(
#             radius=25,
#             location=[p2[-1], p2[0]],
#             color='blue',
#             fill=True,
#         ).add_to(m)
#         geom = {
#             'type': 'Feature',
#             'geometry': {
#                 'type': 'LineString',
#                 'coordinates': [list(p1), list(p1)]
#             },
#             'properties': {
#                 'times': [tm, tm],
#                 'style': {
#                     'color': 'blue',
#                     'opacity': 0.6
#                 },
#                 'icon': 'circle',
#                 'iconstyle': {
#                     'radius': 25,
#                     'fillOpacity': 0.9,
#                     'iconColor': 'red'
#                 }
#             }
#         }    
# #         features.append(geom)
    
    
# #     if item.details.get('dropoff') is not None:
# #         color = 'brown'

#     geom = {
#         'type': 'Feature',
#         'geometry': {
#             'type': 'LineString',
#             'coordinates': [list(p1), list(p2)]
#         },
#         'properties': {
#             'times': [tm, tm],
#             'style': {
#                 'color': color,
#                 'opacity': 0.6
#             },
#             'icon': 'circle',
#             'iconstyle': {
#                 'radius': 7,
#                 'fillOpacity': 0.9,
#                 'iconColor': 'red'
#             }
#         }
#     }

#     features.append(geom)
    
#     prev_pos = p2

#     folium.Circle(
#         radius=100,
#         location=[p1[1], p1[0]],
#         color='green',
#         fill=False,
#     ).add_to(m)
m


In [67]:
# # m = folium.Map(location=[40.76953, -73.98232])

# TimestampedGeoJson(
#     features,
#     period='PT10M',
#     add_last_point=True,
#     auto_play=False, 
#     loop=False, 
#     max_speed=1,
#     loop_button=True,
#     time_slider_drag_update=True
# ).add_to(m)

# m